In [3]:
!pip install netCDF4
!pip install cartopy

In [4]:
import netCDF4 as nc
import os

ModuleNotFoundError: No module named 'netCDF4'

In [ ]:
directories = ["Precipitation", "SEIO", "WTIO"]

In [ ]:
files_size = 0
for directory in directories:
  for file in os.listdir(f"/content/drive/MyDrive/final_project/Final_project_Tomer_and_Ben/Data_Ben_Tomer/data/CMIP6/{directory}"):
      filename = os.fsdecode(file)
      path = f"/content/drive/MyDrive/final_project/Final_project_Tomer_and_Ben/Data_Ben_Tomer/data/CMIP6/{directory}/{filename}"
      files_size += os.path.getsize(path)
      # ds = nc.Dataset(path)
      # print(f"{directory=}, {file=}")
      # print(f"Information about {directory}/{filename}:")
      # print(ds)
      # for dim in ds.dimensions.values():
      #   print(dim)
      # for var in ds.variables:
      #   print(var)
      # print("\n\n")
tot_size = files_size / (1024 * 1024)
print(f"Total size of netCDF data: {tot_size:.2f} MB")

In [ ]:
ds = nc.Dataset(f"/content/drive/MyDrive/final_project/Final_project_Tomer_and_Ben/Data_Ben_Tomer/data/CMIP6/Precipitation/pr_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_19811016-20101016_v20191108.nc")

In [ ]:
ds.metadata

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

data = ds
pr_october = data.variables['pr'][:]
time = data.variables['time']
time_units = time.units
time_calendar = time.calendar

october_indices = np.array([i for i in range(len(time)) if nc.num2date(time[i], units=time_units, calendar=time_calendar).month == 10])
pr_october = pr_october[october_indices]

# Select the latitude and longitude range for the Middle East (25°N to 40°N, 25°E to 40°E)
lat = data.variables['lat'][:]
lon = data.variables['lon'][:]
# lat_indices = np.where((lat >= 25) & (lat <= 40))[0]
# lon_indices = np.where((lon >= 25) & (lon <= 40))[0]

# pr_middle_east = pr_october[:, lat_indices][:, :, lon_indices]
pr_middle_east = pr_october[:, :, :]


# Calculate the average precipitation for each grid point
avg_pr_middle_east = np.sum(pr_middle_east, axis=0)

# Plot the map with the heatmap overlay
plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([25, 40, 25, 40], crs=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, color='lightgray')

plt.imshow(avg_pr_middle_east, origin='lower', cmap='jet', extent=[lon[0], lon[-1], lat[0], lat[-1]],
           transform=ccrs.PlateCarree(), interpolation='nearest')

# Add colorbar
cbar = plt.colorbar(label='Average Precipitation (mm/day)', orientation='horizontal')
cbar.set_label('Average Precipitation (mm/day)')

# Add title
plt.title('Average Precipitation for October in the Middle East')

plt.show()

